In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


For this project, I'm using the Darts library which enables me to deal with TimeSeries easier.

In [2]:
!pip install darts==0.23.1 &> /dev/null

# Data

Firstly, I load all of the files into DataFrames which I then display so I can have an understanding of what i'm dealing with.

In [3]:
#### Load all Datasets
df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
df_holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
df_sample_submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

**train.csv**

In [4]:
display(df_train.head(5))

id        date  store_nbr      family  sales  onpromotion
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0
1   1  2013-01-01          1   BABY CARE    0.0            0
2   2  2013-01-01          1      BEAUTY    0.0            0
3   3  2013-01-01          1   BEVERAGES    0.0            0
4   4  2013-01-01          1       BOOKS    0.0            0

> Here are the columns of df_train:
>
> * id – the index of the row
> * date – the current date
> * store_nbr – the store
> * family – the product family
> * sales – number of sales in this family
> * onpromotion – the number of products on promotion in this family

**holidays_events.csv**

It might not be directly related to the stores, however it affects sales.

In [5]:
display(df_holidays_events.head(5))

date     type    locale locale_name                    description  \
0  2012-03-02  Holiday     Local       Manta             Fundacion de Manta   
1  2012-04-01  Holiday  Regional    Cotopaxi  Provincializacion de Cotopaxi   
2  2012-04-12  Holiday     Local      Cuenca            Fundacion de Cuenca   
3  2012-04-14  Holiday     Local    Libertad      Cantonizacion de Libertad   
4  2012-04-21  Holiday     Local    Riobamba      Cantonizacion de Riobamba   

   transferred  
0        False  
1        False  
2        False  
3        False  
4        False

> Here are the columns in the holiday_events dataframe:
> 
> * date – the date of the holiday
> * type – the type of holiday (Holiday, Event, Transfer (see transferred column), Additional, Bridge, Work Day)
> * locale – the scope of the event (Local, Regional, National)
> * locale_name – the city where the event takes place
> * description – name of the event
> * transferred – whether the event has been transferred (moved to another day) or not

**oil.csv**

Daily oil price from January 01, 2013 to August 31, 2017

In [6]:
display(df_oil.head(5))

date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20

**stores.csv**

Information about the 54 stores.

In [7]:
display(df_stores.head(5))

store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4

> Here are the columns in the stores dataframe:
> 
> * store_nbr – the store
> * city – the city where the store is located
> * state – the state where the store is located
> * type – the type of the store
> * cluster – the number of similar stores in the vicinity

**transactions.csv**

The number of daily transactions by store.

In [8]:
display(df_transactions.head(5))

date  store_nbr  transactions
0  2013-01-01         25           770
1  2013-01-02          1          2111
2  2013-01-02          2          2358
3  2013-01-02          3          3487
4  2013-01-02          4          1922

**test.csv**

The test dataframe is identical to the train dataframe but it misses the sales column which we have to predict. The file starts on August 16, 2017 and ends on August 31, 2017. We also have the sample_submission.csv to fill in with the number of sales per day and per family:

In [9]:
display(df_test.head(5))
display(df_sample_submission.head(5))

id        date  store_nbr      family  onpromotion
0  3000888  2017-08-16          1  AUTOMOTIVE            0
1  3000889  2017-08-16          1   BABY CARE            0
2  3000890  2017-08-16          1      BEAUTY            2
3  3000891  2017-08-16          1   BEVERAGES           20
4  3000892  2017-08-16          1       BOOKS            0

id  sales
0  3000888    0.0
1  3000889    0.0
2  3000890    0.0
3  3000891    0.0
4  3000892    0.0

> Here are the columns of df_test:
>
> * id – the index of the row
> * date – the current date
> * store_nbr – the store
> * family – the product family
> * onpromotion – the number of products on promotion in this family

# Preprocessing

In order to make things simpler for my models I created some covariances from the original data.

In [10]:
family_list = df_train['family'].unique()
store_list = df_stores['store_nbr'].unique()
display(family_list)
display(store_list)

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54])

Prepare Data Frames, merging train and stores and then sorting it by store number, product family and date. This will give easier access to the models.

In [11]:
train_merged = pd.merge(df_train, df_stores, on = 'store_nbr')
train_merged = train_merged.sort_values(['store_nbr', 'family', 'date'])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})
display(train_merged.head(5))

id        date store_nbr      family  sales  onpromotion   city  \
0        0  2013-01-01         1  AUTOMOTIVE    0.0            0  Quito   
1782  1782  2013-01-02         1  AUTOMOTIVE    2.0            0  Quito   
3564  3564  2013-01-03         1  AUTOMOTIVE    3.0            0  Quito   
5346  5346  2013-01-04         1  AUTOMOTIVE    3.0            0  Quito   
7128  7128  2013-01-05         1  AUTOMOTIVE    5.0            0  Quito   

          state type cluster  
0     Pichincha    D      13  
1782  Pichincha    D      13  
3564  Pichincha    D      13  
5346  Pichincha    D      13  
7128  Pichincha    D      13

Same thing for the test DataFrame.

In [12]:
df_test_dropped = df_test.drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr','family'])

display(df_test_sorted.head())

id        date  store_nbr      family
0     3000888  2017-08-16          1  AUTOMOTIVE
1782  3002670  2017-08-17          1  AUTOMOTIVE
3564  3004452  2017-08-18          1  AUTOMOTIVE
5346  3006234  2017-08-19          1  AUTOMOTIVE
7128  3008016  2017-08-20          1  AUTOMOTIVE

**Strategy**

I could use a global model applied on the 1782 (33 x 52) time series. I could also use a model for each time series. Both models have disadvantages though with the first being too general and the second too specific. For this reason we will try to approach the problem with an in between strategy fitting a model for each product family, so 33 models trained on 54 time series.

In [13]:
import darts
from darts import TimeSeries

In [14]:
##Creating subsets per family in a dictionary
family_dict = {}
for family in family_list:
    df_family = train_merged.loc[train_merged['family'] == family]
    TS_family_list = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col = 'date',
                                group_cols = ['store_nbr', 'family'],
                                static_cols = ['city', 'state', 'type', 'cluster'],
                                value_cols = 'sales',
                                fill_missing_dates = True, ##fill missing dates
                                freq = 'D')  ##interval days
    for ts in TS_family_list:
        ts = ts.astype(np.float32)
    
    ##Sorting by store number
    TS_family_list = sorted(TS_family_list, key = lambda ts: int(ts.static_covariates_values()[0][0]))
    family_dict[family] = TS_family_list

This how the first timeseries looks like for one of the product families.

In [15]:
display(family_dict['BEAUTY'][0])

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[2.]],

       [[0.]],

       ...,

       [[1.]],

       [[6.]],

       [[4.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr  family   city      state...
    hierarchy:          None

We retrieve all the values indicated above: the number of sales, the date of each sale in Coordinates > date, and the dependent covariates in Attributes > static_covariates.

**Normalizing Data**

Before fitting our models on the timeseries it's a good practice to normalize them. It will enhance the models' performance. For this we'll use some of the built in functions of the DARTS library.

In [16]:
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
import sklearn

In [17]:
family_pipeline_dict = {}
family_transformed_dict = {}
##For each product family
for key in family_dict:
    ##Filling missing values
    train_filler = MissingValuesFiller(verbose = False, n_jobs = -1, name = 'Fill NaNs')
    ##One hot encoder for static covariates
    static_cov_transformer = StaticCovariatesTransformer(verbose = False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name = 'Encoder')
    ##Applying log transformation
    log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform") 
    ##Scaling time series
    train_scaler = Scaler(verbose = False, n_jobs = -1, name = 'Scale')
    ##Pipeline the above transformers
    train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])
    train_transformed = train_pipeline.fit_transform(family_dict[key])
    
    ##Store pipeline and transformed time series for each product family
    family_pipeline_dict[key] = train_pipeline
    family_transformed_dict[key] = train_transformed

Congragulations! We now have our main time series ready, normalized and one hot encoded.

**Extra covariates (time series features)**

**Date**

I want to squeeze as much information as possible from the date. 

In [18]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

##Duration of dataset
full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')


year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

Example of how our time covariant looks like.

In [19]:
display(print(time_cov.components.values))
display(time_cov[100])

['year' 'month' 'day' 'dayofyear' 'dayofweek' 'weekofyear'
 'linear_increase']


None

/opt/conda/lib/python3.10/site-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (time: 1, component: 7, sample: 1)> Size: 28B
array([[[2013.],
        [   4.],
        [  11.],
        [ 101.],
        [   3.],
        [  15.],
        [ 100.]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 8B 2013-04-11
  * component  (component) object 56B 'year' 'month' ... 'linear_increase'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

Now, I need to normalize this covariant as well.

In [20]:
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
##Split before that dates to be predicted
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

**Oil**

I know the oil prices in advance (future covariate). To help the models we will extract the moving average of the oil prices so we can highlight potential trends.

In [21]:
from darts.models import MovingAverage
# Oil Price

oil = TimeSeries.from_dataframe(df_oil, 
                                time_col = 'date', 
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price of 7 and 28 days
oil_moving_average_7 = MovingAverage(window=7)
oil_moving_average_28 = MovingAverage(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

This is what it looks like for index 100.

In [22]:
display(oil_moving_averages[100])

/opt/conda/lib/python3.10/site-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 2, sample: 1)> Size: 8B
array([[[0.78416604],
        [0.78240633]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-04-11
  * component  (component) object 16B 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

**Holidays**

Ferdinand Berr in his notebook has implemented functions to detail these holidays. In particular, he adds information about whether the holiday is Christmas day, whether it is a soccer game day, etc.

In [23]:
##Specify the type of holiday
def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):
            
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
            
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

In [24]:
##Remove zeros and duplicates
def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):
            
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')
            
            ##Check for zeros
            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            ##Check for duplicates
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

In [25]:
##Store to holiday connection
def holiday_TS_list_54(holiday_list):

    listofseries = []
    
    for i in range(0,54):
            
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

Apply the functions and normalize.

In [26]:
list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

This is what it looks like for index 100 for the first store

In [27]:
display(len(holidays_transformed))
display(holidays_transformed[0].components.values)
display(holidays_transformed[0][100])

54

array(['national_holiday', 'earthquake_relief', 'christmas',
       'football_event', 'national_event', 'work_day', 'local_holiday'],
      dtype=object)

/opt/conda/lib/python3.10/site-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 7, sample: 1)> Size: 28B
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-04-11
  * component  (component) object 56B 'national_holiday' ... 'local_holiday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

**Promotion**

Extracting time series for each product family of the 54 stores.

In [28]:
df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
  df_family = df_promotion.loc[df_promotion['family'] == family]

  list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')
  
  for ts in list_of_TS_promo:
    ts = ts.astype(np.float32)

  family_promotion_dict[family] = list_of_TS_promo

This is what looks like for the first store of index 100 and family AUTOMOTIVE.

In [29]:
display(family_promotion_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1704, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 8B 'onpromotion'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\ncomponent  ...
    hierarchy:          None

Diving a bit deeper we can calculate the moving averages as we did for the oil price.

In [30]:
from tqdm import tqdm

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
  promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])
  
  promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])
  
  # Moving Averages for Promotion Family Dictionaries
  promo_moving_average_7 = MovingAverage(window=7)
  promo_moving_average_28 = MovingAverage(window=28)

  promotion_covs = []

  for ts in promotion_transformed:
    ma_7 = promo_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())  
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
    ma_28 = promo_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())  
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
    promo_and_mas = ts.stack(ma_7).stack(ma_28)
    promotion_covs.append(promo_and_mas)

  promotion_transformed_dict[key] = promotion_covs

100%|██████████| 33/33 [01:09<00:00,  2.11s/it]


This is what it looks like for the first index of the family AUTOMOTIVE

In [31]:
display(promotion_transformed_dict['AUTOMOTIVE'][0].components.values)
display(promotion_transformed_dict['AUTOMOTIVE'][0][1])

array(['onpromotion', 'promotion_ma_7', 'promotion_ma_28'], dtype=object)

/opt/conda/lib/python3.10/site-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 3, sample: 1)> Size: 24B
array([[[0.],
        [0.],
        [0.]]])
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-01-02
  * component  (component) object 24B 'onpromotion' ... 'promotion_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\nglobal_comp...
    hierarchy:          None

**Covariates grouping**

Now that we finished with fututre covariates let's group them in the same timeseries, so it's easier to process.

Starting with the dates, the oil price and its moving averages.

In [32]:
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

Then for each store we get the TimeSeries of the holidays with the general covariates.

In [33]:
store_covariates_future = []

for store in range(0,len(store_list)):
  stacked_covariates = holidays_transformed[store].stack(general_covariates)  
  store_covariates_future.append(stacked_covariates)

Lastly, we combine these with the promotion TimeSeries of each family of each store.

In [34]:
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future

100%|██████████| 33/33 [00:06<00:00,  5.30it/s]


So, we end up with this future covariates TimeSeries.

In [35]:
display(future_covariates_dict['AUTOMOTIVE'][0].components)

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28'],
      dtype='object', name='component')

**Transactions**

Extracting TimeSeries from transactions and normalizing them.

In [36]:
##Extracting TimeSeries
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

In [37]:
from datetime import datetime, timedelta

transactions_list_full = []

##Filling missing values
for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    ts = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions")
    transactions_list_full.append(ts)

##Normalizing
transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

This is what it looks like for the first store.

In [38]:
display(transactions_transformed[0])

<TimeSeries (DataArray) (time: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.        ]],

       [[0.69831293]],

       [[0.60635131]],

       ...,

       [[0.13761164]],

       [[0.57327158]],

       [[0.5600397 ]]])
Coordinates:
  * time       (time) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'transactions'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

Now our preprocessing is done.

# Modeling

Preparing the training data for each product family since we are going to train a model for each one. This model will be the LightGBM which seems to perform well. 

In [39]:
from darts.models import LightGBMModel

LGBM_Models_Submission = {}

display("Training...")

for family in tqdm(family_list):
  ##Extracting TimeSeries per family and intersecting with future covariates.
  sales_family = family_transformed_dict[family]
  training_data = [ts for ts in sales_family] 
  TCN_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]
  
  ##Tunning hyperparameters
  LGBM_Model_Submission = LightGBMModel(lags = 63,
                                        lags_future_covariates = (14,1),
                                        lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22],
                                        output_chunk_length=1,
                                        random_state=2024,
                                        gpu_use_dp= "false",
                                        )

  LGBM_Model_Submission.fit(series=train_sliced, 
                        future_covariates=TCN_covariates,
                        past_covariates=transactions_transformed)
  ##Storing trained models
  LGBM_Models_Submission[family] = LGBM_Model_Submission

'Training...'

  0%|          | 0/33 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.525932 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43593
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.461126


  3%|▎         | 1/33 [00:19<10:31, 19.72s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.287176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28071
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.026578


...

 94%|█████████▍| 31/33 [09:46<00:39, 19.91s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.312909 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46005
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.087959


 97%|█████████▋| 32/33 [10:00<00:18, 18.03s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.402739 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43458
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.512093


100%|██████████| 33/33 [10:20<00:00, 18.81s/it]


In the above code, we only use lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22]. Because during the 16th prediction (the one of August 31, 2017), the values of the past covariates from -1 to -15 are not known.

After training, we obtain 33 Machine Learning models stored in LGBM_Models_Submission.

**Prediction**

Using these stored models we can now do our predictions.

In [40]:
LGBM_Forecasts_Families_Submission = {}

for family in tqdm(family_list):

  sales_family = family_transformed_dict[family]
  training_data = [ts for ts in sales_family]
  LGBM_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                         series=train_sliced,
                                         future_covariates=LGBM_covariates,
                                         past_covariates=transactions_transformed)
  
  LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

100%|██████████| 33/33 [00:30<00:00,  1.07it/s]


Since our data was scaled we need to inverse transform them now for the submission to make sense.

In [41]:
LGBM_Forecasts_Families_back_Submission = {}

for family in tqdm(family_list):

  LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

  0%|          | 0/33 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 33/33 [00:19<00:00,  1.74it/s]


Now lets go from predicted TimeSeries to predicted DataFrame

In [42]:
for family in tqdm(LGBM_Forecasts_Families_back_Submission):
  for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
    if (family_dict[family][n].univariate_values()[-21:] == 0).all():
        LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

listofseries = []

for store in tqdm(range(0,54)):
  for family in family_list:
      oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
      oneforecast.columns = ['fcast']
      listofseries.append(oneforecast)

df_forecasts = pd.concat(listofseries) 
df_forecasts.reset_index(drop=True, inplace=True)

# No Negative Forecasts
df_forecasts[df_forecasts < 0] = 0
forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

# Submission
submission_kaggle = forecasts_kaggle_sorted

100%|██████████| 54/54 [00:02<00:00, 24.85it/s]


This how the predictions look like:

In [43]:
submission_kaggle.head()

id        sales
0  3000888     3.137218
1  3000889     0.000000
2  3000890     3.962451
3  3000891  2276.543092
4  3000892     0.039745

**Ensemble**

We created our first submission, however we want to use multiple models and get the best prediction.
I'll train another 3 models with the following parameters.

In [50]:
model_params = [
    {"lags" : 7, "lags_future_covariates" : (16,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 365, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 730, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}
]

For each of these we will train 33 models.

In [45]:
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from lightgbm import early_stopping

submission_kaggle_list = []

for params in model_params:

  LGBM_Models_Submission = {}

  display("Training...")

  for family in tqdm(family_list):

    # Define Data for family
    sales_family = family_transformed_dict[family]
    training_data = [ts for ts in sales_family] 
    TCN_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                          lags_future_covariates = params["lags_future_covariates"],
                                          lags_past_covariates = params["lags_past_covariates"],
                                          output_chunk_length=1,
                                          random_state=2024,
                                          gpu_use_dp= "false")
      
    LGBM_Model_Submission.fit(series=train_sliced, 
                          future_covariates=TCN_covariates,
                          past_covariates=transactions_transformed)

    LGBM_Models_Submission[family] = LGBM_Model_Submission
    
  display("Predictions...")


  LGBM_Forecasts_Families_Submission = {}

  for family in tqdm(family_list):

    sales_family = family_transformed_dict[family]
    training_data = [ts for ts in sales_family]
    LGBM_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                          series=train_sliced,
                                          future_covariates=LGBM_covariates,
                                          past_covariates=transactions_transformed)
    
    LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

  # Transform Back

  LGBM_Forecasts_Families_back_Submission = {}

  for family in tqdm(family_list):

    LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

  # Prepare Submission in Correct Format

  for family in tqdm(LGBM_Forecasts_Families_back_Submission):
    for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
      if (family_dict[family][n].univariate_values()[-21:] == 0).all():
          LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)
          
  listofseries = []

  for store in tqdm(range(0,54)):
    for family in family_list:
        oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
        oneforecast.columns = ['fcast']
        listofseries.append(oneforecast)

  df_forecasts = pd.concat(listofseries) 
  df_forecasts.reset_index(drop=True, inplace=True)

  # No Negative Forecasts
  df_forecasts[df_forecasts < 0] = 0
  forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

  # Submission
  submission_kaggle_list.append(forecasts_kaggle_sorted)


In [59]:
submission_kaggle_list

[            id        sales
 0      3000888     3.419391
 1      3000889     0.000000
 2      3000890     3.848118
 3      3000891  2191.396295
 4      3000892     0.010238
 ...        ...          ...
 28507  3029395   325.976791
 28508  3029396   102.224190
 28509  3029397  1265.511962
 28510  3029398   104.591132
 28511  3029399    12.832998
 
 [28512 rows x 2 columns]]

In [57]:

df_sample_submission['sales'] = (submission_kaggle[['sales']]+submission_kaggle_list[0][['sales']]+submission_kaggle_list[1][['sales']]+submission_kaggle_list[2][['sales']])/4

In [58]:
df_sample_submission.to_csv('/kaggle/working/submission.csv', index=False)